# 1. Data collection

## 1.1 Collect recall data from cpsc.gov

In [1]:
%%bash
    scrapy crawl CpscSpider -o downloads/cpsc.csv 2> downloads/cpsc_log.txt

## 1.2 Collect company data from compustat

In [1]:
print('placeholder')

placeholder


In [2]:
import wrds_tools

In [3]:
import parameters

wrds = wrds_tools.WrdsConnection(wrds_username=parameters.wrds_username)

/home/julian/PycharmProjects/corporate_disruptions/venv/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Loading library list...
Done


In [4]:
from datetime import date

wrds.set_observation_period(start_date=date(year=2009, month=1, day=1), 
                            end_date=date(year=2018, month=12, day=31))

In [5]:
wrds.build_sp500()

In [6]:
wrds.add_industry_classifiers(get_GICS=True)

In [7]:
wrds.add_names()

In [8]:
wrds.dataset.head()

,gvkey,SIC,NAICS,GICS_group,GICS_industry,GICS_sector,GICS_subindustry,name
0,001045,4512,481111,2030,203020,20,20302010,AMERICAN AIRLINES GROUP INC
1,001075,4911,2211,5510,551010,55,55101010,PINNACLE WEST CAPITAL CORP
2,001078,2834,325412,3510,351010,35,35101010,ABBOTT LABORATORIES
3,001161,3674,334413,4530,453010,45,45301020,ADVANCED MICRO DEVICES
4,001177,6324,524114,3510,351020,35,35102030,AETNA INC


### 1.2.1 Filter by industry

We use the GICS industry classification system to filter. For an overview, see https://en.wikipedia.org/wiki/Global_Industry_Classification_Standard . Industry Group 2550 is Retailing.

In [9]:
wrds.filter_by_industry(industry_code='2550', classification_system='GICS_group')

In [10]:
print('Number of observations: ', len(wrds.dataset))

wrds.dataset.head()

Number of observations:  42


,index,gvkey,SIC,NAICS,GICS_group,GICS_industry,GICS_sector,GICS_subindustry,name
0,41,002184,5731,443142,2550,255040,25,25504020,BEST BUY CO INC
1,83,003813,5331,452990,2550,255030,25,25503020,TARGET CORP
2,89,004016,5331,452990,2550,255030,25,25503020,DOLLAR GENERAL CORP
3,112,004560,5331,452990,2550,255030,25,25503020,FAMILY DOLLAR STORES
4,115,004611,5311,452111,2550,255030,25,25503010,MACY'S INC
